# Cirq Fundamentals for Quantum Computing

This notebook covers the fundamental concepts of quantum computing using Google's Cirq library.

In [ ]:
import cirq
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional
import sympy

## 1. Qubits and Quantum States

In [ ]:
qubit = cirq.LineQubit(0)
print(f"Single qubit: {qubit}")

qubits = cirq.LineQubit.range(3)
print(f"Multiple qubits: {qubits}")

grid_qubits = cirq.GridQubit.square(2)
print(f"Grid qubits: {grid_qubits}")

## 2. Basic Quantum Gates

In [ ]:
def demonstrate_basic_gates():
    q0, q1 = cirq.LineQubit.range(2)
    
    circuit = cirq.Circuit()
    
    circuit.append([
        cirq.H(q0),
        cirq.X(q1),
        cirq.Y(q0),
        cirq.Z(q1),
        cirq.S(q0),
        cirq.T(q1),
    ])
    
    circuit.append(cirq.CNOT(q0, q1))
    
    circuit.append([
        cirq.rx(np.pi/4)(q0),
        cirq.ry(np.pi/3)(q1),
        cirq.rz(np.pi/6)(q0),
    ])
    
    print("Basic quantum gates circuit:")
    print(circuit)
    return circuit

basic_circuit = demonstrate_basic_gates()

## 3. Quantum Circuit Construction

In [ ]:
def create_bell_state():
    q0, q1 = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()
    
    circuit.append([
        cirq.H(q0),
        cirq.CNOT(q0, q1)
    ])
    
    print("Bell state preparation:")
    print(circuit)
    return circuit

def create_ghz_state(n_qubits: int = 3):
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    
    circuit.append(cirq.H(qubits[0]))
    
    for i in range(n_qubits - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
    
    print(f"GHZ state for {n_qubits} qubits:")
    print(circuit)
    return circuit

bell_circuit = create_bell_state()
ghz_circuit = create_ghz_state(4)

## 4. Quantum Fourier Transform

In [ ]:
def quantum_fourier_transform(qubits: List[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()
    n = len(qubits)
    
    for i in range(n):
        circuit.append(cirq.H(qubits[i]))
        
        for j in range(i + 1, n):
            angle = 2 * np.pi / (2 ** (j - i + 1))
            circuit.append(cirq.CZPowGate(exponent=angle/np.pi)(qubits[i], qubits[j]))
    
    for i in range(n // 2):
        circuit.append(cirq.SWAP(qubits[i], qubits[n - i - 1]))
    
    return circuit

qft_qubits = cirq.LineQubit.range(4)
qft_circuit = quantum_fourier_transform(qft_qubits)
print("Quantum Fourier Transform:")
print(qft_circuit)

## 5. Grover's Algorithm

In [ ]:
def oracle(qubits: List[cirq.Qid], marked_state: int) -> cirq.Circuit:
    circuit = cirq.Circuit()
    
    for i, bit in enumerate(bin(marked_state)[2:].zfill(len(qubits))):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    
    circuit.append(cirq.Z.controlled(len(qubits) - 1)(*qubits))
    
    for i, bit in enumerate(bin(marked_state)[2:].zfill(len(qubits))):
        if bit == '0':
            circuit.append(cirq.X(qubits[i]))
    
    return circuit

def diffusion_operator(qubits: List[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()
    
    circuit.append([cirq.H(q) for q in qubits])
    circuit.append([cirq.X(q) for q in qubits])
    
    circuit.append(cirq.Z.controlled(len(qubits) - 1)(*qubits))
    
    circuit.append([cirq.X(q) for q in qubits])
    circuit.append([cirq.H(q) for q in qubits])
    
    return circuit

def grover_algorithm(n_qubits: int, marked_state: int) -> cirq.Circuit:
    qubits = cirq.LineQubit.range(n_qubits)
    circuit = cirq.Circuit()
    
    circuit.append([cirq.H(q) for q in qubits])
    
    n_iterations = int(np.pi / 4 * np.sqrt(2**n_qubits))
    
    for _ in range(n_iterations):
        circuit.append(oracle(qubits, marked_state))
        circuit.append(diffusion_operator(qubits))
    
    circuit.append(cirq.measure(*qubits, key='result'))
    
    return circuit

grover_circuit = grover_algorithm(3, 5)
print("Grover's Algorithm for 3 qubits, searching for state |101>:")
print(grover_circuit)

## 6. Variational Quantum Eigensolver (VQE)

In [ ]:
def create_ansatz(qubits: List[cirq.Qid], depth: int = 2) -> Tuple[cirq.Circuit, List[sympy.Symbol]]:
    circuit = cirq.Circuit()
    symbols = []
    
    for d in range(depth):
        for i, qubit in enumerate(qubits):
            symbol = sympy.Symbol(f'theta_{d}_{i}')
            symbols.append(symbol)
            circuit.append(cirq.ry(symbol)(qubit))
        
        for i in range(len(qubits) - 1):
            circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
    
    return circuit, symbols

def create_hamiltonian_circuit(qubits: List[cirq.Qid]) -> cirq.PauliSum:
    return (
        -1.0 * cirq.Z(qubits[0]) * cirq.Z(qubits[1]) +
        0.5 * cirq.X(qubits[0]) +
        0.5 * cirq.X(qubits[1])
    )

vqe_qubits = cirq.LineQubit.range(2)
ansatz_circuit, params = create_ansatz(vqe_qubits)
hamiltonian = create_hamiltonian_circuit(vqe_qubits)

print("VQE Ansatz Circuit:")
print(ansatz_circuit)
print(f"\nParameters: {params}")
print(f"\nHamiltonian: {hamiltonian}")

## 7. Quantum Circuit Simulation

In [ ]:
def simulate_circuit(circuit: cirq.Circuit, repetitions: int = 1000):
    simulator = cirq.Simulator()
    
    if any(isinstance(op.gate, cirq.MeasurementGate) for op in circuit.all_operations()):
        result = simulator.run(circuit, repetitions=repetitions)
        print("Measurement results:")
        print(result.histogram(key='result'))
        return result
    else:
        result = simulator.simulate(circuit)
        print("Final state vector:")
        print(np.round(result.final_state_vector, 3))
        return result

print("Simulating Bell state:")
bell_result = simulate_circuit(bell_circuit)

print("\nSimulating Grover's algorithm:")
grover_result = simulate_circuit(grover_circuit)

## 8. Quantum Error Correction

In [ ]:
def bit_flip_code_encoder(data_qubit: cirq.Qid, ancilla_qubits: List[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()
    
    for ancilla in ancilla_qubits:
        circuit.append(cirq.CNOT(data_qubit, ancilla))
    
    return circuit

def bit_flip_code_decoder(data_qubit: cirq.Qid, ancilla_qubits: List[cirq.Qid], 
                          syndrome_qubits: List[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()
    
    circuit.append(cirq.CNOT(data_qubit, syndrome_qubits[0]))
    circuit.append(cirq.CNOT(ancilla_qubits[0], syndrome_qubits[0]))
    
    circuit.append(cirq.CNOT(ancilla_qubits[0], syndrome_qubits[1]))
    circuit.append(cirq.CNOT(ancilla_qubits[1], syndrome_qubits[1]))
    
    circuit.append(cirq.measure(*syndrome_qubits, key='syndrome'))
    
    return circuit

def apply_error(qubits: List[cirq.Qid], error_prob: float = 0.1) -> cirq.Circuit:
    circuit = cirq.Circuit()
    
    for qubit in qubits:
        if np.random.random() < error_prob:
            circuit.append(cirq.X(qubit))
    
    return circuit

data_q = cirq.LineQubit(0)
ancilla_q = cirq.LineQubit.range(1, 3)
syndrome_q = cirq.LineQubit.range(3, 5)

error_correction_circuit = cirq.Circuit()
error_correction_circuit.append(bit_flip_code_encoder(data_q, ancilla_q))
error_correction_circuit.append(apply_error([data_q] + ancilla_q))
error_correction_circuit.append(bit_flip_code_decoder(data_q, ancilla_q, syndrome_q))

print("Bit-flip error correction circuit:")
print(error_correction_circuit)

## 9. Quantum Phase Estimation

In [ ]:
def quantum_phase_estimation(unitary_gate: cirq.Gate, n_precision_qubits: int = 3) -> cirq.Circuit:
    precision_qubits = cirq.LineQubit.range(n_precision_qubits)
    target_qubit = cirq.LineQubit(n_precision_qubits)
    
    circuit = cirq.Circuit()
    
    circuit.append([cirq.H(q) for q in precision_qubits])
    
    circuit.append(cirq.X(target_qubit))
    
    for i, control_qubit in enumerate(precision_qubits):
        controlled_u = unitary_gate.controlled()
        for _ in range(2**i):
            circuit.append(controlled_u(control_qubit, target_qubit))
    
    inverse_qft = quantum_fourier_transform(precision_qubits).inverse()
    circuit.append(inverse_qft)
    
    circuit.append(cirq.measure(*precision_qubits, key='phase'))
    
    return circuit

phase_gate = cirq.Z**(1/4)
qpe_circuit = quantum_phase_estimation(phase_gate)
print("Quantum Phase Estimation circuit:")
print(qpe_circuit)

## 10. Quantum Teleportation

In [ ]:
def quantum_teleportation() -> cirq.Circuit:
    alice_data = cirq.LineQubit(0)
    alice_bell = cirq.LineQubit(1)
    bob_bell = cirq.LineQubit(2)
    
    circuit = cirq.Circuit()
    
    circuit.append([
        cirq.ry(np.pi/4)(alice_data),
        cirq.rz(np.pi/6)(alice_data)
    ])
    
    circuit.append([
        cirq.H(alice_bell),
        cirq.CNOT(alice_bell, bob_bell)
    ])
    
    circuit.append([
        cirq.CNOT(alice_data, alice_bell),
        cirq.H(alice_data)
    ])
    
    circuit.append([
        cirq.measure(alice_data, key='m1'),
        cirq.measure(alice_bell, key='m2')
    ])
    
    circuit.append([
        cirq.X(bob_bell).with_classical_controls('m2'),
        cirq.Z(bob_bell).with_classical_controls('m1')
    ])
    
    return circuit

teleportation_circuit = quantum_teleportation()
print("Quantum Teleportation circuit:")
print(teleportation_circuit)

## 11. Noise Modeling and Mitigation

In [ ]:
def add_noise_to_circuit(circuit: cirq.Circuit, noise_prob: float = 0.01) -> cirq.Circuit:
    noisy_circuit = cirq.Circuit()
    
    for moment in circuit:
        noisy_circuit.append(moment)
        
        for op in moment:
            if not isinstance(op.gate, cirq.MeasurementGate):
                for qubit in op.qubits:
                    noisy_circuit.append(cirq.depolarize(noise_prob)(qubit))
    
    return noisy_circuit

def simulate_with_noise(circuit: cirq.Circuit, noise_prob: float = 0.01, repetitions: int = 1000):
    noisy_circuit = add_noise_to_circuit(circuit, noise_prob)
    
    simulator = cirq.DensityMatrixSimulator()
    
    if any(isinstance(op.gate, cirq.MeasurementGate) for op in circuit.all_operations()):
        result = simulator.run(noisy_circuit, repetitions=repetitions)
        return result
    else:
        result = simulator.simulate(noisy_circuit)
        return result

test_circuit = create_bell_state()
test_circuit.append(cirq.measure(*cirq.LineQubit.range(2), key='result'))

print("Original circuit:")
clean_result = simulate_circuit(test_circuit)

print("\nNoisy circuit simulation:")
noisy_result = simulate_with_noise(test_circuit, noise_prob=0.05)
print(noisy_result.histogram(key='result'))

## 12. Custom Gate Implementation

In [ ]:
class CustomControlledGate(cirq.Gate):
    def __init__(self, angle: float):
        self.angle = angle
    
    def _num_qubits_(self) -> int:
        return 2
    
    def _unitary_(self):
        c = np.cos(self.angle / 2)
        s = np.sin(self.angle / 2)
        return np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, c, -1j * s],
            [0, 0, -1j * s, c]
        ])
    
    def _circuit_diagram_info_(self, args):
        return ['@', f'R({self.angle:.2f})']

def test_custom_gate():
    qubits = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()
    
    custom_gate = CustomControlledGate(np.pi / 3)
    
    circuit.append([
        cirq.H(qubits[0]),
        custom_gate(*qubits),
        cirq.measure(*qubits, key='result')
    ])
    
    print("Circuit with custom gate:")
    print(circuit)
    
    return circuit

custom_circuit = test_custom_gate()
simulate_circuit(custom_circuit)